In [111]:
# for Spotipy
import spotipy # don't forget to write: pip install spotipy on gitbash or cmd or whatever
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import json
import sys

# Regular python
import pandas as pd
import time # make computer go to sleep
import random  # for random numbers
import datetime as dt

In [3]:
# Parse JSON
data = {
    "clientID" : "bc9f3e2fa5994153b808dec97aadb7c8",
    "clientSecret" : "ffb21ad5ba2d486ba533d42aa4091892" 
}
with open('authorization.json', 'w') as outfile:
    json.dump(data, outfile)

In [7]:
# Reading JSON file as a dataframe
# clients = pd.read_json("authorization.json")

# Reading JSON file
# Python Parsing JSON 
with open('authorization.json') as json_file:
    data = json.load(json_file)
data # shows contents of json

{'clientID': 'bc9f3e2fa5994153b808dec97aadb7c8',
 'clientSecret': 'ffb21ad5ba2d486ba533d42aa4091892'}

In [2]:
myURI = "spotify:playlist:4UzXak95ei1TL0VddWOWni"

data = [
    {
        "uri" : "spotify:playlist:4UzXak95ei1TL0VddWOWni",
        "like": True
    },
    {
        "uri" : "spotify:playlist:4Fa86B4AHUbsFDSqfCq3Wo",
        "like" : False
    },
    {
        "uri" : "spotify:artist:7JJFJc0AqAeSKo0MeAmEMe",
        "like" : True
    }
]
with open("testSpotify.json", "w") as outfile:
    json.dump(data, outfile)
    
# Getting my credentials
credentials = json.load(open('authorization.json'))
clientID = credentials['clientID']
clientSecret = credentials['clientSecret']

playlistIndex = 2
# Opening testSpotify json file and reading
# basically, getting the first playlist (Boys Love)
playlists = json.load(open("testSpotify.json"))
playlistURI = playlists[playlistIndex]["uri"]
like = playlists[playlistIndex]["like"]


In [124]:
# Setting up manager and client or something..
clientCredManager = SpotifyClientCredentials(client_id = clientID, client_secret = clientSecret)


credentials = oauth2.SpotifyClientCredentials(
        client_id= clientID,
        client_secret = clientSecret)

token = credentials.get_access_token()
sp = spotipy.Spotify(client_credentials_manager = clientCredManager, auth=token)
#spotify = spotipy.Spotify(auth=token)

<ipython-input-124-7084fb1531e8>:9: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = credentials.get_access_token()


In [143]:
# IDEA: maybe have an artist class or something...
# IDEA: or maybe save in dictionary
# key = artist, value = id


def getTrackID(artist, track):
    '''
    Helper method to get the track id from a given artist and track/song
    
    @param string artist the artist name
    @param string track the song name
    @return 
    
    '''
    query = toQuery(artist = artist, track = track)
#     trackList = sp.search(q = query, type = "track")
    trackID = None
    try:
        trackID = sp.search(q = query, type = "track", limit = 5)["tracks"]["items"][0]["id"]
    except (IndexError):
        print("Oops! Couldn't find ID for song = ", track, " by ", artist)
    return trackID
    


def getArtistID(artist, playlistURI = None):
    '''
    Gets the artist ID.
    First time overloading in Python
    
    @param string artist the artist name
    @param string playlistURI the URI for spotify playlist, if given
    @return int the artist ID from spotify
    '''
    artistID = sp.search(q = artist, type = "artist", limit = 5)["artists"]["items"][0]["id"]    
    return artistID

def toQuery(artist, track):
    '''
    Transforms the artist and track into a query (to later search for trackID)
    
    @param string artist the artist name
    @param string track the track/song name
    @return a Spotify-esque query
    '''
    query = artist + " " + track
    return query


def getAudioFeatures(trackID, track = None):
    '''
    Gets the audio features of a track
    
    @param trackID the track ID
    @param track (optional) the track name
    @return a dictionary of all the audio features
    
    '''
    audioFeature = sp.audio_features(trackID)[0]
    audioFeature.pop("id", None)
    audioFeature.pop("uri", None)
    audioFeature.pop("track_href", None)
    audioFeature.pop("analysis_url", None)
    audioFeature.pop("type", None)
    
    numBars = len(sp.audio_analysis(trackID)["bars"])
    numSections = len(sp.audio_analysis(trackID)["sections"])
    numSegments = len(sp.audio_analysis(trackID)["segments"])
    audioFeature["bars"] = numBars
    
    return audioFeature
    

singer = "train"
# song = "如何"
song = "drive by"
trackID = getTrackID(song, singer)
#print(trackID)
#sp.search(q = "top 100 hits UK", type = "track", limit = 5)["tracks"]
# audioFeature = sp.audio_features(trackID)[0]
audioFeatures = getAudioFeatures(trackID)
audioFeatures
#len(sp.audio_analysis(trackID)["bars"])

{'danceability': 0.765,
 'energy': 0.837,
 'key': 1,
 'loudness': -3.113,
 'mode': 0,
 'speechiness': 0.032,
 'acousticness': 0.00107,
 'instrumentalness': 1.06e-05,
 'liveness': 0.0801,
 'valence': 0.721,
 'tempo': 122.028,
 'duration_ms': 195973,
 'time_signature': 4,
 'bars': 94}

In [ ]:
# IDEA: maybe keep a set of all the artists from UMD...then search up all their songs. 
# TODO: 

In [56]:
scope = "user-library-read"
playlists = sp.user_playlists("spotify")
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

   1 spotify:playlist:37i9dQZF1DXcBWIGoYBM5M Today's Top Hits
   2 spotify:playlist:37i9dQZF1DX0XUsuxWHRQd RapCaviar
   3 spotify:playlist:37i9dQZF1DX1lVhptIYRda Hot Country
   4 spotify:playlist:37i9dQZF1DX10zKzsJ2jva ¡Viva Latino!
   5 spotify:playlist:37i9dQZF1DX4JAvHpjipBk New Music Friday
   6 spotify:playlist:37i9dQZF1DX4sWSpwq3LiO Peaceful Piano
   7 spotify:playlist:37i9dQZF1DX4SBhb3fqCJd Are & Be
   8 spotify:playlist:37i9dQZF1DWXRqgorJj26U Rock Classics
   9 spotify:playlist:37i9dQZF1DX4dyzvuaRJ0n mint
  10 spotify:playlist:37i9dQZF1DXcF6B6QPhFDv Rock This
  11 spotify:playlist:37i9dQZF1DWXJfnUiYjUKT New Music Friday
  12 spotify:playlist:37i9dQZF1DXcRXFNfZr7Tp just hits
  13 spotify:playlist:37i9dQZF1DX4o1oenSJRJd All Out 00s
  14 spotify:playlist:37i9dQZF1DXbTxeAdrVG2l All Out 90s
  15 spotify:playlist:37i9dQZF1DX4UtSsGT1Sbe All Out 80s
  16 spotify:playlist:37i9dQZF1DWTJ7xPn4vNaz All Out 70s
  17 spotify:playlist:37i9dQZF1DXaKIA8E7WcJj All Out 60s
  18 spotify:playlist:37i

 151 spotify:playlist:37i9dQZF1DX37CqNpZIOOZ This Is Richard Strauss
 152 spotify:playlist:37i9dQZF1DX5ZPNS2aXuN4 This Is Bartók
 153 spotify:playlist:37i9dQZF1DX1tT8vLysk8r This Is Prokofiev
 154 spotify:playlist:37i9dQZF1DWSQeR9oTmnyj This Is: John Williams
 155 spotify:playlist:37i9dQZF1DX5M3XSYkKZCl This Is: Pavarotti
 156 spotify:playlist:37i9dQZF1DXaQQM8OxjxoC This Is Brahms
 157 spotify:playlist:37i9dQZF1DX6Hln2awUHXX This Is Wagner
 158 spotify:playlist:37i9dQZF1DWVTUSNpU0pKJ This Is Satie
 159 spotify:playlist:37i9dQZF1DWZhMbcax6CUd This Is Ravel
 160 spotify:playlist:37i9dQZF1DX30FdZ6jR5Gl This Is Philip Glass
 161 spotify:playlist:37i9dQZF1DWTBNkoD4HmYd This Is Vivaldi
 162 spotify:playlist:37i9dQZF1DX93wmY669mVj This Is Liszt
 163 spotify:playlist:37i9dQZF1DWXnRw5VvmZDb This Is Debussy
 164 spotify:playlist:37i9dQZF1DWY3VlkBR4Jhb This Is Schubert
 165 spotify:playlist:37i9dQZF1DWSL7w3snV7Gw This Is: Glenn Gould
 166 spotify:playlist:37i9dQZF1DX9l01QzlK1Yo This Is: London Sy

 301 spotify:playlist:37i9dQZF1DWWhBhYl3ZMvY This Is Rihanna
 302 spotify:playlist:37i9dQZF1DX8IAL5KfmFJA This is Roxette
 303 spotify:playlist:37i9dQZF1DX86aPVQf2emM This Is Sampha
 304 spotify:playlist:37i9dQZF1DX5WbJFtYTzv7 This Is: Skepta
 305 spotify:playlist:37i9dQZF1DWWUJdr9ahsbf This Is Spice Girls
 306 spotify:playlist:37i9dQZF1DX8yMCqd4JdaS This is Sting
 307 spotify:playlist:37i9dQZF1DX5joH2aOAS4r This Is Take That
 308 spotify:playlist:37i9dQZF1DX9AFOBBZSMWw This Is Stormzy
 309 spotify:playlist:37i9dQZF1DX4UK2Qu9x6KC This Is Tina Turner
 310 spotify:playlist:37i9dQZF1DX4c84KflUhSt This Is Tony Bennett
 311 spotify:playlist:37i9dQZF1DX4cEqBpR9k1C This Is Trentemøller
 312 spotify:playlist:37i9dQZF1DX8kI525v95Yu This Is U2
 313 spotify:playlist:37i9dQZF1DWUHyTgDVMaG1 This Is Van Morrison
 314 spotify:playlist:37i9dQZF1DWYrXu9rm1zUZ This Is Volbeat
 315 spotify:playlist:37i9dQZF1DXbW9wx2hgf03 This is: Wiley
 316 spotify:playlist:37i9dQZF1DX7mMRxgkst0Y This Is The Fall
 317 sp

 451 spotify:playlist:4MLFX9OXy9ncfBlTfei9Ok Nightly Meditations
 452 spotify:playlist:71WgbCkbVXGzAltJTAT6lD (Descent) A Beginning
 453 spotify:playlist:37i9dQZF1DX1s9knjP51Oa Calm Vibes
 454 spotify:playlist:37i9dQZF1DWZhzMp90Opmn Pink Noise
 455 spotify:playlist:37i9dQZF1DX9if5QDLdzCa Ocean Escapes
 456 spotify:playlist:37i9dQZF1DWSW4ppn40bal Sleep Noise
 457 spotify:playlist:37i9dQZF1DWVEt8B7a1H1M Birds in the Forest
 458 spotify:playlist:37i9dQZF1DX7Mv4BQ3Z4nC The Yule Log
 459 spotify:playlist:0jJQDST7bzwAYlN19tFLXw Covered in Country
 460 spotify:playlist:37i9dQZF1DX5FuBDzVtEFX Hawaiian Dreams
 461 spotify:playlist:0mY9BQvlpraYLzg9BfKuVa Traffic Jams
 462 spotify:playlist:2nb7lCySvvf7PNxHUD5oun #JazzAppreciationMonth
 463 spotify:playlist:2f4rXblveCedKdU2adFgC8 Motivation Montage
 464 spotify:playlist:7MsOqrEv1TxiHPDiYlWVLx Sugar & Spice
 465 spotify:playlist:3TymQFosH9UgfP5jV5mPEe Soul Soothers
 466 spotify:playlist:1JuraqRjLX4CmhyvvuXiPF Winding Roads
 467 spotify:playlist:0Na

 601 spotify:playlist:31agQ1yW51tGsDIFZbAZMl XFM JUNE 26
 602 spotify:playlist:1omLAKnMAhhkMSe2IGisUZ The Beach Boys: The Real Best Of...
 603 spotify:playlist:0kfqBZx9obUKiLVDwTqGly boys cast
 604 spotify:playlist:2G5Wlqt6clkHH2KFK2LS5Q Happy Birthday George!
 605 spotify:playlist:6vpXSVefNo73POjhwyiWmu Remembering 2Pac
 606 spotify:playlist:3rMnCpY0WEROlje2b8Bjdn The Hot Sounds of 1926
 607 spotify:playlist:45Cyxm3R3Gpn2upYrjcPbc Happy Birthday Kanye!
 608 spotify:playlist:0dVRyMERU2g9M4tWEfaD5B Mollie King 
 609 spotify:playlist:5Ir6gs0eXKyPj3Qin8uEun Happy Birthday Sir Tom!
 610 spotify:playlist:44uPoVqT6tOn5AphWVnSvu Happy Birthday Prince!
 611 spotify:playlist:66uSq0mmMKat8vPRxsbmhp Happy Birthday Joe Cocker
 612 spotify:playlist:4njfFyb4EgzewhH9kTQuwN Happy Birthday Mollie!
 613 spotify:playlist:2vfyCK7WsXnVeItNwuiFFR Congratulations, Sofia and Daniel
 614 spotify:playlist:5zpAWvvDe0Q8TXjrgInIke Welcome Baby Kimye!
 615 spotify:playlist:52FOMIrumGaNXnwKfrlAVf The Roots of... Vam

 751 spotify:playlist:37i9dQZF1DWTypZHlgEy1G Standards de la Chanson Française
 752 spotify:playlist:37i9dQZF1DX6sa8XjvdJIf This Is New Found Glory
 753 spotify:playlist:37i9dQZF1DX9E92APFiTvV This Is Rush
 754 spotify:playlist:37i9dQZF1DX5kVmFQxhnLW This Is Michael Bublé
 755 spotify:playlist:37i9dQZF1DWVpFvorxSZ6K This Is Leonard Cohen
 756 spotify:playlist:37i9dQZF1DX6KE9M7aTCxK This Is NOFX
 757 spotify:playlist:37i9dQZF1DXdJa941ExayM Punk Unleashed
 758 spotify:playlist:37i9dQZF1DX3LDIBRoaCDQ Classic Punk
 759 spotify:playlist:37i9dQZF1DXa9wYJr1oMFq Pop Punk Powerhouses
 760 spotify:playlist:37i9dQZF1DX3MU5XUozve7 Skatepark Punks
 761 spotify:playlist:37i9dQZF1DX7WJ4yDmRK8R The Essential Ska
 762 spotify:playlist:37i9dQZF1DX0KpeLFwA3tO New Punk Tracks
 763 spotify:playlist:37i9dQZF1DX3YlUroplxjF Crash Course
 764 spotify:playlist:37i9dQZF1DXd6tJtr4qeot Punk Essentials
 765 spotify:playlist:37i9dQZF1DX1zG0qRnyaQ3 Political Punks
 766 spotify:playlist:37i9dQZF1DXb2cYklA89lm Hot Rods

 901 spotify:playlist:37i9dQZF1DWVxpHBekDUXK Coping With Loss
 902 spotify:playlist:37i9dQZF1DX9LT7r8qPxfa Noir
 903 spotify:playlist:37i9dQZF1DX6Oqe1LC2A2s Next Classical
 904 spotify:playlist:37i9dQZF1DWYZW76Iw4Lij Medieval 50: Spotify Picks
 905 spotify:playlist:37i9dQZF1DX8ONeLSIu0Ai Modernism 50: Spotify Picks
 906 spotify:playlist:37i9dQZF1DX1FuwERxHNIM Date Night Classical
 907 spotify:playlist:37i9dQZF1DX5kUFcXez8kt Classical Intimate Dinner
 908 spotify:playlist:37i9dQZF1DXcbK8HZQNxAq Indie Classical: Composed In 21st Century
 909 spotify:playlist:37i9dQZF1DX5hL1aT2vhMb Women of Classical
 910 spotify:playlist:37i9dQZF1DX10uaP8FoDNe Minimalism
 911 spotify:playlist:37i9dQZF1DX5AviTMPF0km Contemporary Chinese Classical
 912 spotify:playlist:37i9dQZF1DWV7cvDzE3MOI This Is Chopin
 913 spotify:playlist:1M2abQkxCwrLjzLGgKWnDP Bellini's Opera
 914 spotify:playlist:37i9dQZF1DWSXyN2w7ILMl Haunting Choral
 915 spotify:playlist:37i9dQZF1DX0CM2xT3aAkN Essence of the Renaissance
 916 spot

1051 spotify:playlist:37i9dQZF1DX8gDIpdqp1XJ Modern Psychedelia
1052 spotify:playlist:37i9dQZF1DX1fmyMdW2PaL Complete Chaos
1053 spotify:playlist:37i9dQZF1DX0IlCGIUGBsA Ultimate Party Classics
1054 spotify:playlist:37i9dQZF1DX44AF5xq04Oh Reggae BBQ
1055 spotify:playlist:37i9dQZF1DXcTieYAg7jq1 Retro Pop
1056 spotify:playlist:37i9dQZF1DWWOGXILUAh53 Fresh Dance Pop
1057 spotify:playlist:37i9dQZF1DX98f0uoU1Pcs Noisy
1058 spotify:playlist:37i9dQZF1DXauOWFg72pbl Beer & Wings
1059 spotify:playlist:37i9dQZF1DXdTCdwCKzXwo Early Alternative
1060 spotify:playlist:37i9dQZF1DX09NvEVpeM77 Born in the USA
1061 spotify:playlist:37i9dQZF1DWZMxqlsEB7BT Chips and Salsa
1062 spotify:playlist:37i9dQZF1DX1MUPbVKMgJE Disco Forever
1063 spotify:playlist:37i9dQZF1DX8XxBtyUnC7B Souvenirs
1064 spotify:playlist:37i9dQZF1DWZgauS5j6pMv Nu-Funk
1065 spotify:playlist:37i9dQZF1DX23YPJntYMnh Funk Rock
1066 spotify:playlist:37i9dQZF1DX4pUKG1kS0Ac Guilty Pleasures
1067 spotify:playlist:2lOMjzyPMlcIvcaWlaF6MB Historic Ven

1201 spotify:playlist:37i9dQZEVXbJPcfkRz0wJ0 Australia Top 50
1202 spotify:playlist:37i9dQZEVXbKM896FDX8L1 Ireland Top 50
1203 spotify:playlist:37i9dQZEVXbKGcyg6TFGx6 Luxembourg Top 50
1204 spotify:playlist:37i9dQZEVXbIQnj7RRhdSX Italy Top 50
1205 spotify:playlist:37i9dQZEVXbN6itCcaL3Tt Poland Top 50
1206 spotify:playlist:37i9dQZEVXbKyJS56d1pgi Portugal Top 50
1207 spotify:playlist:37i9dQZEVXbLwpL8TjsxOG Hong Kong Top 50
1208 spotify:playlist:37i9dQZEVXbJlfUljuZExa Malaysia Top 50
1209 spotify:playlist:37i9dQZEVXbK4gjvS1FjPY Singapore Top 50
1210 spotify:playlist:37i9dQZEVXbLesry2Qw2xS Estonia Top 50
1211 spotify:playlist:37i9dQZEVXbKMzVsSGQ49S Iceland Top 50
1212 spotify:playlist:37i9dQZEVXbMx56Rdq5lwc Lithuania Top 50
1213 spotify:playlist:37i9dQZEVXbJWuzDrTxbKS Latvia Top 50
1214 spotify:playlist:37i9dQZEVXbO3qyFxbkOE1 Mexico Top 50
1215 spotify:playlist:4lCv0oN3gPxoqvXmA3cDjS Top Picks Toplists
1216 spotify:playlist:3xGWOQUB6jgHd4PLVQvbfG Top Picks Pop
1217 spotify:playlist:0q2dbyU

1351 spotify:playlist:37i9dQZF1DWU8quswnFt3c Amplify: Seventeen Takeover
1352 spotify:playlist:37i9dQZF1DWYtQSOiZF6hj Happy Birthday
1353 spotify:playlist:6PzNMpCPMDMo1pOUjNhDUC Kids B-day Party
1354 spotify:playlist:2mADaSxlIcxRDSG9e9vLlh Walk The Dog
1355 spotify:playlist:0uZg7go8TKu5hpv1L6CJ4y Tagga För OS
1356 spotify:playlist:5wPUMOWXlhmk9XmGO4RtKz Commuter Flow (That Fire For Your Aux Cord)
1357 spotify:playlist:37i9dQZF1DWWiDhnQ2IIru 70s Road Trip
1358 spotify:playlist:1cddSHoeIrE3m0i8TqYNbn Annorlunda Kärlekslåtar
1359 spotify:playlist:5Dn14a8hNIhT3NLLjUUBGL Fuck Valentine!
1360 spotify:playlist:1apRF10Iufr0NBBiF1u7E5 Valentine anger
1361 spotify:playlist:7qeSmszUjRBWOwHzlxfEXt Det är så tråkigt och trist utan dig
1362 spotify:playlist:2PMlGn5a2jxukdhCYQyQvG Inget är större än KÄRLEKEN
1363 spotify:playlist:3cC0Hnr7idCemR4JxOh2lI Folk & Rock 200 favoriter
1364 spotify:playlist:24HEIuw8etbkitpDCxZ1dQ The Critics Choice 2012
1365 spotify:playlist:0NYlvGXveeyvhb5G80LPtA EM Fotboll

# Getting All Albums From Artist ID

In [17]:
#playlists = sp.user_playlists("spotify")
# With an artist ID, how to grab all their songs
# Look at the artist: Boy Sompob
artistID = playlistURI.split(":")[2]
artistAlbums = sp.artist_albums(artistID)["items"]
for i, playlist in enumerate(artistAlbums):
    # artist = playlist["artists"][0]["name"]
    print("%4d %s %s" % (i + 1, playlist['uri'],  playlist['name']))
    
# a function
def getAlbums(artistID):
    albums = []
    artistAlbums = sp.artist_albums(artistID)["items"]
    for i, playlist in enumerate(artistAlbums):
        #artist = playlist["artists"][0]["name"]
        
        albums.append(playlist["name"])
        # A helper print line
#         print("%4d %s %s" % (i + 1, playlist['uri'],  playlist['name']))
    return albums

def getFeatures(trackID):
    '''
    A helper method to get the audio features of the song
    
    @param int trackID the Spotify ID of the track/song
    @return the audio features
    
    '''
    # A JSON format
    features = sp.audio_features(trackID)

boySompobAlbums = getAlbums(artistID)

   1 spotify:album:2282fQEMBPsB90KHB1khTY The Luckiest Boy And The Luckiest Fans (Live)
   2 spotify:album:5eMK5WKI1bOKpThYtvlDIW Why I Love You
   3 spotify:album:4bZQoaw4elqZVMiAdhar9w ครั้งนี้ (เพลงประกอบซีรีส์บังเอิญรัก 2 A Chance to love)
   4 spotify:album:3rzEFOUwsYGDeERFtwp5iB เพื่อนเธอ- Live (From " เพลงประกอบซีรีส์บังเอิญรัก 2)
   5 spotify:album:3NoamIlgkZyCBQwQtkQNbH เพื่อนเธอ (Just Friend?) [From "เพลงประกอบซีรีส์บังเอิญรัก 2"]
   6 spotify:album:6FWiRiHZHFc4WZwhOdcanL หวัง
   7 spotify:album:6jGnrA1561aH510qyfJJRz Jigsaw Heart (English Version)
   8 spotify:album:4X2YdBEVqCUuGF5PufmTyG พบเพื่อจาก...รักเพื่อลา (Live session- From "Until we meet again the series")
   9 spotify:album:5mbrSLnrog1oAt5ZUnlHa9 The Luckiest Boy (Original Soundtrack)
  10 spotify:album:4au75uFCaaeKVzbaMsjNTU โชคดีแค่ไหน (Original Soundtrack)
  11 spotify:album:73Ij4BSutAwc0P8e3LdHwE Until We Meet Again (Ost. Until We Meet Again the Series)
  12 spotify:album:3HHInykbtAwdE859nIyJKd พบเพื่อจาก...รัก

'7JJFJc0AqAeSKo0MeAmEMe'

HTTP Error for GET to https://api.spotify.com/v1/me/ returned 401 due to Unauthorized.


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/me/:
 Unauthorized., reason: None